In [ ]:
!pip install -e .

In [1]:
from deeplens.extractor import ExtractSingleSample
from deeplens.intervene import InterveneFeatures, ReinjectSingleSample

In [109]:
extractor = ExtractSingleSample(
    model="gpt2",
    layer=3,
    max_length=1024,
    device="auto"
)

acts = extractor.get_mlp_acts("Once upon a time, there was a")

Running on device: cuda


In [110]:
intervene = InterveneFeatures(
    sae_model=r"saved_models\gpt2L3-untied\run_20251229_102023\best_model.pt",
    sae_config=r"C:\code\deeplens\demo\config.yaml",
    device="cuda"
)

features = intervene.get_alive_features(acts, token_position=-1)
print(features.shape)

Running on device: cuda
torch.Size([552])


In [111]:
og, mod = intervene.intervene_feature(
    activations=acts,
    feature=features[1].item(),
    alpha=100.0,
    token_positions=-1
)

In [124]:
reinject = ReinjectSingleSample(
    hf_model="gpt2",
    device="auto"
)

gen = reinject.reinject_and_generate(
    text="Once upon a time, there was a",
    modified_activations=mod,
    layer=3,
    generate=False,
    max_new_tokens=25,
    temperature=1.0
)

Running on device: cuda


In [127]:
print(gen.squeeze().shape)

torch.Size([8, 50257])
